# Gerando a estrutura organizacional em rede

In [1]:
!pip install igraph
!pip install pycairo
!pip install cairocffi
!pip install python-igraph

In [53]:
import time
import datetime as dt
import pickle
import igraph as ig
print("igraph-version : ",ig.__version__)
import cairo
import pandas as pd
import json

base_path = "C:/projetos/prova-de-conceito/dados/siorg/"

url_base_siorg = "http://estruturaorganizacional.dados.gov.br/doc/"

path = ['estrutura-organizacional-completa.json'
            ,'estrutura-organizacional-resumida.json',
            'orgao-entidade-completa.json','orgao-entidade-resumida.json',
            'estrutura-organizacional-alteracoes.json']

from urllib.request import urlopen

with urlopen(url_base_siorg + 'estrutura-organizacional/resumida') as response:
    body = response.read()

data = json.loads(body)   

for i in data['unidades'][:10]:
    print(i)
    codigos = i['codigoUnidade'].split("/")
    print(codigos[len(codigos) - 1]) 
    print(i['nome'])
    codigosPai = i['codigoUnidadePai'].split("/")
    print(codigosPai[len(codigosPai) - 1])
    print("---------------------")


igraph-version :  0.10.1
{'codigoUnidade': 'http://estruturaorganizacional.dados.gov.br/id/unidade-organizacional/200000', 'codigoUnidadePai': '', 'codigoOrgaoEntidade': 'http://estruturaorganizacional.dados.gov.br/id/unidade-organizacional/200000', 'codigoTipoUnidade': 'http://estruturaorganizacional.dados.gov.br/id/tipo-unidade/ente', 'nome': 'República Federativa do Brasil', 'sigla': ' ', 'codigoEsfera': '', 'codigoPoder': '', 'codigoNaturezaJuridica': None, 'codigoSubNaturezaJuridica': None, 'nivelNormatizacao': 'LEI_DECRETO', 'versaoConsulta': '99.0.0', 'dataInicialVersaoConsulta': '2013-11-11', 'dataFinalVersaoConsulta': None, 'operacao': None, 'codigoUnidadePaiAnterior': None, 'codigoOrgaoEntidadeAnterior': None}
200000
República Federativa do Brasil

---------------------
{'codigoUnidade': 'http://estruturaorganizacional.dados.gov.br/id/unidade-organizacional/200001', 'codigoUnidadePai': 'http://estruturaorganizacional.dados.gov.br/id/unidade-organizacional/200000', 'codigoOrga

In [110]:
g = ig.Graph(directed=True)
g["name"]="Unidades Organizacionais da Administração Pública - Siorg"

start_time = time.time()


for i in data['unidades']:
    try:
        vert = g.vs.find(i['codigoUnidade'])             
    except:
        codigos = i['codigoUnidade'].split("/")         
        g.add_vertices(codigos[len(codigos) - 1])
        vert = g.vs.find(codigos[len(codigos) - 1])
        vert["codigo"] = (codigos[len(codigos) - 1])        
        vert["nome"] = i["nome"]
        codigosPai = i['codigoUnidadePai'].split("/")
        vert["codigoPai"] = codigosPai[len(codigosPai) - 1]
        #print(i['nome'])
        #print("u",codigos[len(codigos) - 1])
        

for i in data['unidades']:
        codigos = i['codigoUnidade'].split("/")         
        codigosPai = i['codigoUnidadePai'].split("/")
        try:            
            vertS = g.vs.find(codigosPai[len(codigosPai) - 1])
            vertT = g.vs.find(codigos[len(codigos) - 1])
            g.add_edges([(vertS, vertT)])
        except:
            None
        

#for vs in g.vs:
#    print(vs)
                
print(g.summary())

with open("grafoSiorg.pkl", 'wb') as pickle_file:
    pickle.dump(g, pickle_file)
    
print("--- %s seconds ---" % (time.time() - start_time))
print("---",(time.time() - start_time)/60, "minutes ---")



IGRAPH DN-- 79583 79582 -- Unidades Organizacionais da Administração Pública - Siorg
+ attr: name (g), codigo (v), codigoPai (v), name (v), nome (v)
--- 338.12905049324036 seconds ---
--- 5.635484174887339 minutes ---


In [76]:
with cairo.SVGSurface("example.svg", 200, 200) as surface:
    context = cairo.Context(surface)
    x, y, x1, y1 = 0.1, 0.5, 0.4, 0.9
    x2, y2, x3, y3 = 0.6, 0.1, 0.9, 0.5
    context.scale(200, 200)
    context.set_line_width(0.04)
    context.move_to(x, y)
    context.curve_to(x1, y1, x2, y2, x3, y3)
    context.stroke()
    context.set_source_rgba(1, 0.2, 0.2, 0.6)
    context.set_line_width(0.02)
    context.move_to(x, y)
    context.line_to(x1, y1)
    context.move_to(x2, y2)
    context.line_to(x3, y3)
    context.stroke()